# 1. WHY DATA PIPELINES ARE IMPORTANT
### CONSISTANTLY REFINING YOUR CRUDE DATA INTO FINE FEATURES IS KEY

Sklearn has a wonderful tool called [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). These are objects that take a list of transformers as parameters through which data will be passed. Several prebuilt transformers are available, such as StandardScalar. Others can be found here: [Sklearn Pipeline Transformers](https://scikit-learn.org/stable/data_transforms.html). 

```python
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier())
    ])
```
However, these prebuilt transformsers can be limiting, or a desired transformation has not been implemented into sklearn. So lets build our own!

At first, the sklearn Pipeline interface and custom transformers can be intimidating, but they don't have to be. 

<img src='images/refinery_pipes.jpg'/>

In [26]:
import pandas as pd
import numpy as np

### Why use pipelines?

Pipelines are useful because they can be trained to process data in a consistant way. 
Here are two examples:
1. Standardizing columns consistently
2. Creating Dummy Variables

#### 1. Standardizing Columns Consistently

In [27]:
# create two fake columns
training_df = pd.DataFrame(np.random.normal(0, 1, size=5), columns=['value'])
test_df = pd.DataFrame(np.random.normal(3, 1, size=5), columns=['value'])

In [28]:
training_df

,value
0,-0.665241
1,0.013470
2,-0.102443
3,1.824287
4,0.857221


In [29]:
test_df

,value
0,4.030714
1,5.098856
2,3.194427
3,3.753700
4,2.037587


If we want to scale`training_column` to put it into a machine learning model, then scale `test_column` to put it into the same machine learning model, the naive approach would be to scale the `training_column` and scale the `testing_column` separately:

In [30]:
training_mean = training_df.loc[:,'value'].mean()
training_standard_deviation = training_df.loc[:,'value'].std()
training_column_standardized = (training_df.loc[:,'value'] - training_mean)/ training_standard_deviation

In [31]:
training_column_standardized

0   -1.081820
1   -0.383006
2   -0.502353
3    1.481444
4    0.485735
Name: value, dtype: float64

In [32]:
test_mean = test_df.loc[:,'value'].mean()
test_standard_deviation = test_df.loc[:,'value'].std()
wrong_test_column_standardized = (test_df.loc[:,'value'] - test_mean)/ test_standard_deviation

In [33]:
wrong_test_column_standardized

0    0.362499
1    1.312318
2   -0.381148
3    0.116171
4   -1.409840
Name: value, dtype: float64

__*What is wrong with this?*__

The testing data is not being standardized in relation to the training data! Valuable information has been destroyed.

Here is the correct way to standardsize the `test_column`

## Dummies

In [34]:
correct_test_column_standardized = (test_df.loc[:,'value'] - training_mean)/ test_standard_deviation

In [35]:
correct_test_column_standardized

0    3.241453
1    4.191272
2    2.497806
3    2.995126
4    1.469115
Name: value, dtype: float64

#### 2. Creating Dummy Variables

In [36]:
training_species_df = pd.DataFrame([['cat'], ['dog'], ['bat'], ['mouse'], ['rat']], columns=['species'])

In [37]:
test_species_df = pd.DataFrame([[np.nan], ['dog'], ['rat'], ['cat'], ['kangaroo']], columns=['species'])

In [38]:
training_species_df

,species
0,cat
1,dog
2,bat
3,mouse
4,rat


In [39]:
test_species_df

,species
0,NaN
1,dog
2,rat
3,cat
4,kangaroo


In [40]:
pd.get_dummies(training_species_df.loc[:'species'])

,species_bat,species_cat,species_dog,species_mouse,species_rat
0,0,1,0,0,0
1,0,0,1,0,0
2,1,0,0,0,0
3,0,0,0,1,0
4,0,0,0,0,1


In [41]:
pd.get_dummies(test_species_df.loc[:'species'])

,species_cat,species_dog,species_kangaroo,species_rat
0,0,0,0,0
1,0,1,0,0
2,0,0,0,1
3,1,0,0,0
4,0,0,1,0


__*What is wrong with this?*__

`pd.get_dummies()` is pretty dumb, huh? 

Our machine learning model will not work correctly with the above dataframes!
- The columns are inconsistent in length! 
- Our columns are not ordered in the same way! 
- Our test_df is missing columns!
- Our test_df has columns that are not in the train_df!

To fix the problems we find in `pd.get_dummies()`, we can create our own code that we can later put into functions or a class.

The following code serves as a demonstration of the concepts we will use later to create a custom Dummifier Class:

In [42]:
# Create an empty dataframe to add columns to:
training_dummy_df = pd.DataFrame()

In [43]:
training_dummy_df

""


In [44]:
# save the unique values found in the training data into a list 
unique_col_values = training_species_df['species'].unique()

In [45]:
# for each item in the list of unique items, make a pd.Series consisting of 0's and 1's 
# where the item is present in the species column of the training_species_df
for item in unique_col_values:
    training_dummy_df[item] = (training_species_df.loc[:,'species'] == item).astype(int)

In [46]:
test_dummy_df = pd.DataFrame()

In [47]:
# for each item in the list of unique items, make a pd.Series consisting of 0's and 1's 
# where the item is present in the species column of the test_species_df
for item in unique_col_values:
    test_dummy_df[item] = (test_species_df.loc[:,'species'] == item).astype(int)

In [49]:
training_dummy_df

,cat,dog,bat,mouse,rat
0,1,0,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,0,1


In [50]:
test_dummy_df

,cat,dog,bat,mouse,rat
0,0,0,0,0,0
1,0,1,0,0,0
2,0,0,0,0,1
3,1,0,0,0,0
4,0,0,0,0,0
